Используем базу транзакций интернет-магазина из Великобритании

Задача: ознакомиться с данными и составить аналитический отчёт.

Отчёт должен включать:
 - Описание данных (что отражают, качество, полнота, какие некорректности и аномалии);
 - Описание примененного подхода к анализу данных (ограничения накладываемые представленными данными и соответствующие интерпретации допущения);
 - Описание бизнес-заказчика, его гипотетических болей и целей развития бизнеса;
 - Описание стейкхолдеров отчёта с обоснованием выбранных метрик (системы метрик);
 - Рекомендации по модели данных бизнес-заказчика (что вы считаете важным отслеживать в дополнение к тому, что уже имеется в датасете);
 - В завершении работы, необходимо сформулировать выводы и составить рекомендации для бизнес-заказчика.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
df = pd.read_excel('Копия Data Sample (AIC) для итоговой работы .xlsx')

In [4]:
df.info() # Не везде есть описание заказа. Есть много заказов, в которых не указан пользователь

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54214 entries, 0 to 54213
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    54214 non-null  object        
 1   StockCode    54214 non-null  object        
 2   Description  54080 non-null  object        
 3   Quantity     54214 non-null  int64         
 4   InvoiceDate  54214 non-null  datetime64[ns]
 5   UnitPrice    54214 non-null  float64       
 6   CustomerID   40643 non-null  float64       
 7   Country      54214 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 3.3+ MB


In [5]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom


In [6]:
df.describe()

,Quantity,UnitPrice,CustomerID
count,54214.000000,54214.000000,40643.000000
mean,9.610802,4.307341,15283.422803
std,57.358422,47.834496,1707.982278
min,-9600.000000,0.000000,12347.000000
25%,1.000000,1.250000,13969.000000
50%,3.000000,2.080000,15152.000000
75%,10.000000,4.130000,16775.000000
max,2400.000000,6930.000000,18287.000000


In [7]:
df.Quantity.unique() # В столбце "количество" есть отрицательные значения

array([    6,    32,     3,     8,    10,     2,     1,     4,    -1,
         100,    12,    24,   144,    48,   -24,     5,    40,    36,
          96,    72,    16,    30,     9,     7,    18,    60,    20,
          -3,   108,    75,   320,    17,  -144,    -2,    -4,    21,
          47,    25,   192,    31,    57,    -6,   -70,   -80,    14,
          15,   120,   480,    33,    22,    49,    -8,    -9,    19,
         -72,   160,    64,   104,    13,    26,    11,   408,    50,
         200,    46,    80,   -77,  -150,  -723,   -32,    23,    84,
         -12,   -18,   -11,   113,    45,    35,    38,   115,   -48,
         -10,    -7,   -20,  -240,   460,   240,   288,   -36,    53,
         216,  -120,   150,   168,   -30,   -40,    54,    28,  1284,
          29,    41,    27,  -310,    78,   -15,   384,    73,    37,
         720,   -50,    -5,  -600,   432,  1200,   -19,  -234,   208,
         -76,   -16,   300,   256,  1500,   407,  2400,  -192,   576,
          44,    70,

In [8]:
df.loc[df.Quantity < 0] # Возможно товары с отрицательным количеством - это возврат товаров. 
# У таких заказов перед номером счета указана буква "C"

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
19,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
30,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
31,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
202,C536548,20957,PORCELAIN HANGING BELL SMALL,-1,2010-12-01 14:33:00,1.45,12472.0,Germany
375,C536642,21463,MIRRORED DISCO BALL,-1,2010-12-02 11:56:00,5.95,14390.0,United Kingdom
...,...,...,...,...,...,...,...,...
53704,C581229,79191C,RETRO PLASTIC ELEPHANT TRAY,-24,2011-12-08 10:14:00,0.85,12558.0,USA
53705,C581229,21507,ELEPHANT BIRTHDAY CARD,-12,2011-12-08 10:14:00,0.42,12558.0,USA
53765,C581322,22666,RECIPE BOX PANTRY YELLOW DESIGN,-1,2011-12-08 11:51:00,2.95,18223.0,United Kingdom
53770,C581330,22959,WRAP CHRISTMAS VILLAGE,-25,2011-12-08 11:57:00,0.42,15877.0,United Kingdom


In [9]:
# Каждая строка - один заказ. Добавим новый столбец с итоговой суммой заказа, т.е. UnitPrice*Quantity
df['revenue'] = df.UnitPrice * df.Quantity

In [10]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom,17.85
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom,22.00


## Смотрим данные по странам

In [11]:
df.Country.unique() # В каких странах оформляют покупки на нашем сайте

array(['United Kingdom', 'Australia', 'Netherlands', 'Norway', 'EIRE',
       'Germany', 'France', 'Switzerland', 'Spain', 'Poland', 'Italy',
       'Belgium', 'Lithuania', 'Japan', 'Portugal', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Finland',
       'Austria', 'Israel', 'Greece', 'Hong Kong', 'Singapore', 'Lebanon',
       'United Arab Emirates', 'Czech Republic', 'Canada', 'Unspecified',
       'Brazil', 'USA', 'European Community', 'Bahrain', 'Malta', 'RSA'],
      dtype=object)

In [12]:
df['Country'].value_counts() # Смотрим количество заказов в разрезе по странам

United Kingdom          49602
Germany                   951
France                    881
EIRE                      787
Spain                     253
Netherlands               234
Belgium                   211
Switzerland               189
Portugal                  152
Australia                 130
Norway                    105
Channel Islands            81
Italy                      77
Finland                    72
Cyprus                     61
Sweden                     50
Austria                    48
Poland                     44
Unspecified                36
Denmark                    35
Japan                      33
Hong Kong                  29
Israel                     27
Singapore                  25
USA                        23
Iceland                    15
Greece                     14
Canada                     13
Malta                       9
Lithuania                   6
United Arab Emirates        5
European Community          5
RSA                         4
Brazil    

In [13]:
# Выведем показатель дохода в разрезе по странам
df_country_revenue = df.groupby('Country')['revenue'].sum()
df_country_revenue

Country
Australia                17072.960
Austria                   1036.270
Bahrain                     17.700
Belgium                   4032.060
Brazil                      79.800
Canada                     214.140
Channel Islands           2877.950
Cyprus                     737.260
Czech Republic              82.200
Denmark                   2058.840
EIRE                     27443.140
European Community          80.900
Finland                   2040.600
France                   19472.990
Germany                  22519.200
Greece                     490.820
Hong Kong                -1704.750
Iceland                    271.790
Israel                     531.720
Italy                     1438.620
Japan                     2001.580
Lebanon                     39.600
Lithuania                  291.600
Malta                      282.100
Netherlands              26267.780
Norway                    3859.900
Poland                     904.930
Portugal                  3442.350
RSA         

In [14]:
# По "Hong Kong" получилось отрицательное значение. Если мы предполагаем, что в первоначальных данных отрицательные значения - 
# это возвраты, то итоговое значение "revenue" не может быть меньше 0. Предположим, что это ошибка в данных и не будем учитывать
# данные по "Hong Kong"

## Смотрим данные по пользователям

In [15]:
df.CustomerID.value_counts() # гриуппируем заказы по уникальным пользователям, смотрим сколько заказов сделал 
# каждый из пользователей 

17841.0    771
14911.0    563
14096.0    546
12748.0    447
14606.0    278
          ... 
13754.0      1
15724.0      1
16872.0      1
12830.0      1
15661.0      1
Name: CustomerID, Length: 3819, dtype: int64

In [16]:
df_CustomerID = df.groupby('CustomerID')['revenue'].sum( ) # Суммарный доход от каждого пользователя
df_CustomerID

CustomerID
12347.0    271.79
12348.0    163.20
12349.0    106.84
12350.0     49.80
12352.0    184.40
            ...  
18278.0     49.50
18281.0     16.50
18282.0     22.95
18283.0    215.30
18287.0    158.88
Name: revenue, Length: 3819, dtype: float64

In [17]:
df_CustomerID_mean = df.groupby('CustomerID')['revenue'].mean() # Средний доход от пользователя
df_CustomerID_mean

CustomerID
12347.0    18.119333
12348.0    81.600000
12349.0    15.262857
12350.0    16.600000
12352.0    23.050000
             ...    
18278.0    24.750000
18281.0    16.500000
18282.0    22.950000
18283.0     3.075714
18287.0    26.480000
Name: revenue, Length: 3819, dtype: float64

### Комментарии
- Можно отобразить на дашборде базовые показатели: итоговое количество заказов, итоговую сумма дохода, количество клиентов (уникальных пользователей). Заказчиком может выступать топ-менеджмент компании

- Можем построить дашборд по базовым показателям (из пункта выше) в динамике по месяцам. Заказчиком опять же может выступать топ-менеджемт. По сути это скорее дополнение к предыдущему пункту

- Можно узнать средний чек, количество заказов, суммарный доход в разрезе по странам. Исходя из этого можно определить в каких странах имеет смысл в первую очередь увеличивать продажи. В том числе понять в продвижение в каких странах имеет смысл инвестировать. Заказчиком может быть финансовый директор/директор по развитию/коммерческий директор. Возможная цель бизнеса - расширение географии продаж

- Можно провести RFM-анализ. С помощью этого мы разобьем нашу базу клиентов на сегменты. После этого можем выстраивать разные маркетинговые коммуникации для каждого сегмента. Заказчиком выступает отдел маркетинга. Можем предоставить им эти сегменты, но для выведения на дашборд эта информация не подходит 

## Для дашборда

In [18]:
# Посчитаем количество заказов, количество клиентов и сумму дохода в разрезе по месяцам

In [19]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom,17.85
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom,22.00


### 2010-12

In [20]:
start_date = '2010-12-01 00:00:00'
df_2010_12_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2010-12-31 23:59:59'
df_2010_12_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [21]:
df_2010_12 = df.loc[(df.InvoiceDate >= df_2010_12_start_date) & (df.InvoiceDate <= df_2010_12_end_date)]
df_2010_12

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
0,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
1,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom,54.08
2,536367,21755,LOVE BUILDING BLOCK WORD,3,2010-12-01 08:34:00,5.95,13047.0,United Kingdom,17.85
3,536368,22914,BLUE COAT RACK PARIS FASHION,3,2010-12-01 08:34:00,4.95,13047.0,United Kingdom,14.85
4,536373,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 09:02:00,2.75,17850.0,United Kingdom,22.00
...,...,...,...,...,...,...,...,...,...
4239,539988,84849B,FAIRY SOAP SOAP HOLDER,1,2010-12-23 16:06:00,1.69,18116.0,United Kingdom,1.69
4240,539989,22727,ALARM CLOCK BAKELIKE RED,1,2010-12-23 16:16:00,3.75,NaN,United Kingdom,3.75
4241,539990,21154,RED RETROSPOT OVEN GLOVE,2,2010-12-23 16:31:00,1.25,NaN,United Kingdom,2.50
4242,539990,84378,SET OF 3 HEART COOKIE CUTTERS,1,2010-12-23 16:31:00,1.25,NaN,United Kingdom,1.25


In [22]:
# количество заказов
len(df_2010_12)

4244

In [23]:
# количество клиентов
len(df_2010_12['CustomerID'].unique())

706

In [24]:
# сумма дохода
df_2010_12.revenue.sum()

76439.74

### 2011-01

In [25]:
start_date = '2011-01-01 00:00:00'
df_2011_01_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-01-31 23:59:59'
df_2011_01_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [26]:
df_2011_01 = df.loc[(df.InvoiceDate >= df_2011_01_start_date) & (df.InvoiceDate <= df_2011_01_end_date)]
df_2011_01

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
4244,539993,20682,RED RETROSPOT CHILDRENS UMBRELLA,6,2011-01-04 10:00:00,3.25,13313.0,United Kingdom,19.50
4245,540003,22193,RED DINER WALL CLOCK,2,2011-01-04 10:37:00,8.50,16875.0,United Kingdom,17.00
4246,540003,84970L,SINGLE HEART ZINC T-LIGHT HOLDER,12,2011-01-04 10:37:00,0.95,16875.0,United Kingdom,11.40
4247,540005,85062,PEARL CRYSTAL PUMPKIN T-LIGHT HLDR,6,2011-01-04 10:38:00,1.65,17315.0,United Kingdom,9.90
4248,540005,21615,4 LAVENDER BOTANICAL DINNER CANDLES,12,2011-01-04 10:38:00,1.25,17315.0,United Kingdom,15.00
...,...,...,...,...,...,...,...,...,...
7810,C542744,22700,BLACK AND WHITE DOG BOWL,-10,2011-01-31 16:32:00,0.85,16191.0,United Kingdom,-8.50
7811,542745,85230B,CINNAMON SCENTED VOTIVE CANDLE,1,2011-01-31 16:47:00,0.29,15881.0,United Kingdom,0.29
7812,542745,21472,LADYBIRD + BEE RAFFIA FOOD COVER,1,2011-01-31 16:47:00,3.75,15881.0,United Kingdom,3.75
7813,542745,22649,STRAWBERRY FAIRY CAKE TEAPOT,1,2011-01-31 16:47:00,4.95,15881.0,United Kingdom,4.95


In [27]:
# количество заказов
len(df_2011_01)

3571

In [28]:
# количество клиентов
len(df_2011_01['CustomerID'].unique())

590

In [29]:
# сумма дохода
df_2011_01.revenue.sum()

64125.780000000006

### 2011-02

In [30]:
start_date = '2011-02-01 00:00:00'
df_2011_02_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-02-28 23:59:59'
df_2011_02_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [31]:
df_2011_02 = df.loc[(df.InvoiceDate >= df_2011_02_start_date) & (df.InvoiceDate <= df_2011_02_end_date)]
df_2011_02

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
7815,542776,21218,RED SPOTTY BISCUIT TIN,6,2011-02-01 08:23:00,3.75,15240.0,United Kingdom,22.50
7816,542777,20654,FIRST CLASS LUGGAGE TAG,12,2011-02-01 08:31:00,1.25,14911.0,EIRE,15.00
7817,542777,22433,WATERING CAN GREEN DINOSAUR,6,2011-02-01 08:31:00,1.95,14911.0,EIRE,11.70
7818,542777,85099B,JUMBO BAG RED RETROSPOT,10,2011-02-01 08:31:00,1.95,14911.0,EIRE,19.50
7819,542777,21559,STRAWBERRY LUNCH BOX WITH CUTLERY,6,2011-02-01 08:31:00,2.55,14911.0,EIRE,15.30
...,...,...,...,...,...,...,...,...,...
10630,545217,85131B,BEADED CRYSTAL HEART GREEN ON STICK,1,2011-02-28 16:59:00,0.83,NaN,United Kingdom,0.83
10631,545217,90013B,BLACK VINTAGE EARRINGS,1,2011-02-28 16:59:00,4.15,NaN,United Kingdom,4.15
10632,545217,16235,RECYCLED PENCIL WITH RABBIT ERASER,1,2011-02-28 16:59:00,0.42,NaN,United Kingdom,0.42
10633,545217,21087,SET/6 POSIES PAPER CUPS,2,2011-02-28 16:59:00,0.83,NaN,United Kingdom,1.66


In [32]:
# количество заказов
len(df_2011_02)

2820

In [33]:
# количество клиентов
len(df_2011_02['CustomerID'].unique())

592

In [34]:
# сумма дохода
df_2011_02.revenue.sum()

47766.84

### 2011-03

In [35]:
start_date = '2011-03-01 00:00:00'
df_2011_03_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-03-31 23:59:59'
df_2011_03_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [36]:
df_2011_03 = df.loc[(df.InvoiceDate >= df_2011_03_start_date) & (df.InvoiceDate <= df_2011_03_end_date)]
df_2011_03

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
10635,545220,17091J,VANILLA INCENSE IN TIN,6,2011-03-01 08:30:00,1.25,14620.0,United Kingdom,7.50
10636,545220,84678,CLASSICAL ROSE SMALL VASE,6,2011-03-01 08:30:00,2.55,14620.0,United Kingdom,15.30
10637,545221,22568,FELTCRAFT CUSHION OWL,4,2011-03-01 08:35:00,3.75,14740.0,United Kingdom,15.00
10638,545221,21422,PORCELAIN ROSE SMALL,16,2011-03-01 08:35:00,0.85,14740.0,United Kingdom,13.60
10639,545221,20979,36 PENCILS TUBE RED RETROSPOT,16,2011-03-01 08:35:00,1.25,14740.0,United Kingdom,20.00
...,...,...,...,...,...,...,...,...,...
14326,548534,22752,SET 7 BABUSHKA NESTING BOXES,2,2011-03-31 17:28:00,8.50,18172.0,United Kingdom,17.00
14327,548534,22556,PLASTERS IN TIN CIRCUS PARADE,12,2011-03-31 17:28:00,1.65,18172.0,United Kingdom,19.80
14328,548541,21164,HOME SWEET HOME METAL SIGN,1,2011-03-31 17:59:00,2.95,14957.0,United Kingdom,2.95
14329,548541,82580,BATHROOM METAL SIGN,1,2011-03-31 17:59:00,0.55,14957.0,United Kingdom,0.55


In [37]:
# количество заказов
len(df_2011_03)

3696

In [38]:
# количество клиентов
len(df_2011_03['CustomerID'].unique())

770

In [39]:
# сумма дохода
df_2011_03.revenue.sum()

73452.62

### 2011-04

In [40]:
start_date = '2011-04-01 00:00:00'
df_2011_04_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-04-30 23:59:59'
df_2011_04_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [41]:
df_2011_04 = df.loc[(df.InvoiceDate >= df_2011_04_start_date) & (df.InvoiceDate <= df_2011_04_end_date)]
df_2011_04

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
14331,548550,22423,REGENCY CAKESTAND 3 TIER,3,2011-04-01 08:22:00,12.75,18161.0,United Kingdom,38.25
14332,548551,22904,CALENDAR PAPER CUT DESIGN,6,2011-04-01 08:56:00,2.95,14886.0,United Kingdom,17.70
14333,548551,22286,"DECORATION , WOBBLY RABBIT , METAL",12,2011-04-01 08:56:00,1.65,14886.0,United Kingdom,19.80
14334,548551,22281,EASTER TREE YELLOW BIRDS,2,2011-04-01 08:56:00,5.95,14886.0,United Kingdom,11.90
14335,548552,21936,RED RETROSPOT PICNIC BAG,5,2011-04-01 09:06:00,2.95,17613.0,United Kingdom,14.75
...,...,...,...,...,...,...,...,...,...
17295,551507,21212,PACK OF 72 RETROSPOT CAKE CASES,24,2011-04-28 18:11:00,0.55,18272.0,United Kingdom,13.20
17296,C551510,47566,PARTY BUNTING,-100,2011-04-28 19:43:00,4.15,16163.0,United Kingdom,-415.00
17297,C551513,22667,RECIPE BOX RETROSPOT,-1,2011-04-28 19:52:00,2.95,16592.0,United Kingdom,-2.95
17298,C551513,21232,STRAWBERRY CERAMIC TRINKET BOX,-3,2011-04-28 19:52:00,1.25,16592.0,United Kingdom,-3.75


In [42]:
# количество заказов
len(df_2011_04)

2969

In [43]:
# количество клиентов
len(df_2011_04['CustomerID'].unique())

683

In [44]:
# сумма дохода
df_2011_04.revenue.sum()

49826.89

### 2011-05

In [45]:
start_date = '2011-05-01 00:00:00'
df_2011_05_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-05-31 23:59:59'
df_2011_05_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [46]:
df_2011_05 = df.loc[(df.InvoiceDate >= df_2011_05_start_date) & (df.InvoiceDate <= df_2011_05_end_date)]
df_2011_05

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
17300,551515,20749,ASSORTED COLOUR MINI CASES,12,2011-05-01 10:51:00,6.35,15606.0,United Kingdom,76.20
17301,551516,23159,SET OF 5 PANCAKE DAY MAGNETS,12,2011-05-01 11:10:00,2.08,14800.0,United Kingdom,24.96
17302,551516,21790,VINTAGE SNAP CARDS,1,2011-05-01 11:10:00,0.85,14800.0,United Kingdom,0.85
17303,551516,23256,CHILDRENS CUTLERY SPACEBOY,5,2011-05-01 11:10:00,4.15,14800.0,United Kingdom,20.75
17304,551516,22941,CHRISTMAS LIGHTS 10 REINDEER,12,2011-05-01 11:10:00,8.50,14800.0,United Kingdom,102.00
...,...,...,...,...,...,...,...,...,...
20956,555150,22906,12 MESSAGE CARDS WITH ENVELOPES,1,2011-05-31 15:53:00,3.29,NaN,United Kingdom,3.29
20957,555150,22979,PANTRY WASHING UP BRUSH,1,2011-05-31 15:53:00,3.29,NaN,United Kingdom,3.29
20958,555150,23084,RABBIT NIGHT LIGHT,1,2011-05-31 15:53:00,4.13,NaN,United Kingdom,4.13
20959,555150,84050,PINK HEART SHAPE EGG FRYING PAN,5,2011-05-31 15:53:00,4.13,NaN,United Kingdom,20.65


In [47]:
# количество заказов
len(df_2011_05)

3661

In [48]:
# количество клиентов
len(df_2011_05['CustomerID'].unique())

835

In [49]:
# сумма дохода
df_2011_05.revenue.sum()

68884.0

### 2011-06

In [50]:
start_date = '2011-06-01 00:00:00'
df_2011_06_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-06-30 23:59:59'
df_2011_06_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [51]:
df_2011_06 = df.loc[(df.InvoiceDate >= df_2011_06_start_date) & (df.InvoiceDate <= df_2011_06_end_date)]
df_2011_06

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
20961,555158,22273,FELTCRAFT DOLL MOLLY,6,2011-06-01 08:38:00,2.95,14842.0,United Kingdom,17.70
20962,555158,22355,CHARLOTTE BAG SUKI DESIGN,10,2011-06-01 08:38:00,0.85,14842.0,United Kingdom,8.50
20963,555158,21154,RED RETROSPOT OVEN GLOVE,10,2011-06-01 08:38:00,1.25,14842.0,United Kingdom,12.50
20964,555158,22352,LUNCH BOX WITH CUTLERY RETROSPOT,6,2011-06-01 08:38:00,2.55,14842.0,United Kingdom,15.30
20965,555158,22907,PACK OF 20 NAPKINS PANTRY DESIGN,12,2011-06-01 08:38:00,0.85,14842.0,United Kingdom,10.20
...,...,...,...,...,...,...,...,...,...
24608,558636,22457,NATURAL SLATE HEART CHALKBOARD,3,2011-06-30 19:58:00,2.95,14702.0,United Kingdom,8.85
24609,558636,21468,BUTTERFLY CROCHET FOOD COVER,3,2011-06-30 19:58:00,3.75,14702.0,United Kingdom,11.25
24610,558636,84755,COLOUR GLASS T-LIGHT HOLDER HANGING,8,2011-06-30 19:58:00,0.65,14702.0,United Kingdom,5.20
24611,558637,22436,12 COLOURED PARTY BALLOONS,20,2011-06-30 20:08:00,0.65,17891.0,United Kingdom,13.00


In [52]:
# количество заказов
len(df_2011_06)

3652

In [53]:
# количество клиентов
len(df_2011_06['CustomerID'].unique())

775

In [54]:
# сумма дохода
df_2011_06.revenue.sum()

76033.76

### 2011-07

In [55]:
start_date = '2011-07-01 00:00:00'
df_2011_07_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-07-31 23:59:59'
df_2011_07_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [56]:
df_2011_07 = df.loc[(df.InvoiceDate >= df_2011_07_start_date) & (df.InvoiceDate <= df_2011_07_end_date)]
df_2011_07

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
24613,558638,22674,FRENCH TOILET SIGN BLUE METAL,12,2011-07-01 08:16:00,1.25,16317.0,United Kingdom,15.00
24614,558638,22413,METAL SIGN TAKE IT OR LEAVE IT,6,2011-07-01 08:16:00,2.95,16317.0,United Kingdom,17.70
24615,558639,23206,LUNCH BAG APPLE DESIGN,10,2011-07-01 08:55:00,1.65,13492.0,Switzerland,16.50
24616,558640,21086,SET/6 RED SPOTTY PAPER CUPS,12,2011-07-01 08:58:00,0.65,14911.0,EIRE,7.80
24617,558641,22670,FRENCH WC SIGN BLUE METAL,12,2011-07-01 09:19:00,1.25,17865.0,United Kingdom,15.00
...,...,...,...,...,...,...,...,...,...
28522,561901,22961,JAM MAKING SET PRINTED,288,2011-07-31 15:42:00,1.45,14156.0,EIRE,417.60
28523,561902,22907,PACK OF 20 NAPKINS PANTRY DESIGN,96,2011-07-31 15:50:00,0.85,17404.0,Sweden,81.60
28524,561902,21906,PHARMACIE FIRST AID TIN,32,2011-07-31 15:50:00,6.75,17404.0,Sweden,216.00
28525,561902,22952,60 CAKE CASES VINTAGE CHRISTMAS,240,2011-07-31 15:50:00,0.55,17404.0,Sweden,132.00


In [57]:
# количество заказов
len(df_2011_07)

3914

In [58]:
# количество клиентов
len(df_2011_07['CustomerID'].unique())

758

In [59]:
# сумма дохода
df_2011_07.revenue.sum()

73621.98000000001

### 2011-08

In [60]:
start_date = '2011-08-01 00:00:00'
df_2011_08_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-08-31 23:59:59'
df_2011_08_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [61]:
df_2011_08 = df.loc[(df.InvoiceDate >= df_2011_08_start_date) & (df.InvoiceDate <= df_2011_08_end_date)]
df_2011_08

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
28527,561906,21240,BLUE POLKADOT CUP,16,2011-08-01 10:03:00,0.85,12921.0,United Kingdom,13.6
28528,561906,21154,RED RETROSPOT OVEN GLOVE,10,2011-08-01 10:03:00,1.25,12921.0,United Kingdom,12.5
28529,561906,21212,PACK OF 72 RETROSPOT CAKE CASES,24,2011-08-01 10:03:00,0.55,12921.0,United Kingdom,13.2
28530,561906,20971,PINK BLUE FELT CRAFT TRINKET BOX,12,2011-08-01 10:03:00,1.25,12921.0,United Kingdom,15.0
28531,561907,22621,TRADITIONAL KNITTING NANCY,12,2011-08-01 10:08:00,1.65,14060.0,United Kingdom,19.8
...,...,...,...,...,...,...,...,...,...
32121,565067,22646,CERAMIC STRAWBERRY CAKE MONEY BANK,2,2011-08-31 17:16:00,1.45,15856.0,United Kingdom,2.9
32122,565067,22741,FUNKY DIVA PEN,48,2011-08-31 17:16:00,0.85,15856.0,United Kingdom,40.8
32123,565069,20665,NaN,-49,2011-08-31 17:18:00,0.00,NaN,United Kingdom,-0.0
32124,565071,84754,NaN,-242,2011-08-31 17:21:00,0.00,NaN,United Kingdom,-0.0


In [62]:
# количество заказов
len(df_2011_08)

3599

In [63]:
# количество клиентов
len(df_2011_08['CustomerID'].unique())

733

In [64]:
# сумма дохода
df_2011_08.revenue.sum()

65897.26

### 2011-09

In [65]:
start_date = '2011-09-01 00:00:00'
df_2011_09_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-09-30 23:59:59'
df_2011_09_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [66]:
df_2011_09 = df.loc[(df.InvoiceDate >= df_2011_09_start_date) & (df.InvoiceDate <= df_2011_09_end_date)]
df_2011_09

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
32126,565080,20677,PINK POLKADOT BOWL,8,2011-09-01 08:25:00,1.25,13509.0,United Kingdom,10.00
32127,565087,22943,CHRISTMAS LIGHTS 10 VINTAGE BAUBLES,3,2011-09-01 10:03:00,4.95,13276.0,United Kingdom,14.85
32128,565088,23436,VINTAGE CHRISTMAS GIFT BAG LARGE,12,2011-09-01 10:11:00,1.25,16145.0,United Kingdom,15.00
32129,565088,23437,50'S CHRISTMAS GIFT BAG LARGE,12,2011-09-01 10:11:00,1.25,16145.0,United Kingdom,15.00
32130,565088,20914,SET/5 RED RETROSPOT LID GLASS BOWLS,6,2011-09-01 10:11:00,2.95,16145.0,United Kingdom,17.70
...,...,...,...,...,...,...,...,...,...
37153,569202,21745,GAOLERS KEYS DECORATIVE GARDEN,1,2011-09-30 17:22:00,8.29,NaN,United Kingdom,8.29
37154,569202,21749,LARGE RED RETROSPOT WINDMILL,1,2011-09-30 17:22:00,4.13,NaN,United Kingdom,4.13
37155,569202,21813,GARLAND WITH STARS AND BELLS,2,2011-09-30 17:22:00,3.29,NaN,United Kingdom,6.58
37156,569202,21822,GLITTER CHRISTMAS TREE WITH BELLS,4,2011-09-30 17:22:00,1.63,NaN,United Kingdom,6.52


In [67]:
# количество заказов
len(df_2011_09)

5032

In [68]:
# количество клиентов
len(df_2011_09['CustomerID'].unique())

1045

In [69]:
# сумма дохода
df_2011_09.revenue.sum()

95635.62100000001

### 2011-10

In [70]:
start_date = '2011-10-01 00:00:00'
df_2011_10_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-10-31 23:59:59'
df_2011_10_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [71]:
df_2011_10 = df.loc[(df.InvoiceDate >= df_2011_10_start_date) & (df.InvoiceDate <= df_2011_10_end_date)]
df_2011_10

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
37158,569204,23284,DOORMAT KEEP CALM AND COME IN,15,2011-10-02 10:43:00,7.08,16591.0,United Kingdom,106.20
37159,569205,23400,SHELF WITH 4 HOOKS HOME SWEET HOME,4,2011-10-02 10:55:00,6.25,16923.0,United Kingdom,25.00
37160,569205,23393,HOME SWEET HOME CUSHION COVER,5,2011-10-02 10:55:00,3.75,16923.0,United Kingdom,18.75
37161,569205,22899,CHILDREN'S APRON DOLLY GIRL,2,2011-10-02 10:55:00,2.10,16923.0,United Kingdom,4.20
37162,569205,21258,VICTORIAN SEWING BOX LARGE,2,2011-10-02 10:55:00,12.75,16923.0,United Kingdom,25.50
...,...,...,...,...,...,...,...,...,...
43113,573665,23569,TRADTIONAL ALPHABET STAMP SET,12,2011-10-31 16:24:00,4.95,13982.0,United Kingdom,59.40
43114,573665,23301,GARDENERS KNEELING PAD KEEP CALM,1,2011-10-31 16:24:00,1.65,13982.0,United Kingdom,1.65
43115,573666,21014,SWISS CHALET TREE DECORATION,24,2011-10-31 16:24:00,0.29,15977.0,United Kingdom,6.96
43116,573666,84029E,RED WOOLLY HOTTIE WHITE HEART.,2,2011-10-31 16:24:00,4.25,15977.0,United Kingdom,8.50


In [72]:
# количество заказов
len(df_2011_10)

5960

In [73]:
# количество клиентов
len(df_2011_10['CustomerID'].unique())

1133

In [74]:
# сумма дохода
df_2011_10.revenue.sum()

102977.85

### 2011-11

In [75]:
start_date = '2011-11-01 00:00:00'
df_2011_11_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-11-30 23:59:59'
df_2011_11_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [76]:
df_2011_11 = df.loc[(df.InvoiceDate >= df_2011_11_start_date) & (df.InvoiceDate <= df_2011_11_end_date)]
df_2011_11

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
43118,573744,21915,RED HARMONICA IN BOX,12,2011-11-01 08:16:00,1.25,17733.0,United Kingdom,15.00
43119,573744,22865,HAND WARMER OWL DESIGN,12,2011-11-01 08:16:00,2.10,17733.0,United Kingdom,25.20
43120,573744,23414,ZINC BOX SIGN HOME,12,2011-11-01 08:16:00,8.49,17733.0,United Kingdom,101.88
43121,573745,22617,BAKING SET SPACEBOY DESIGN,3,2011-11-01 08:53:00,4.95,17419.0,United Kingdom,14.85
43122,573745,22845,VINTAGE CREAM CAT FOOD CONTAINER,2,2011-11-01 08:53:00,6.35,17419.0,United Kingdom,12.70
...,...,...,...,...,...,...,...,...,...
51676,579885,23406,HOME SWEET HOME KEY HOLDER,1,2011-11-30 17:37:00,6.25,15444.0,United Kingdom,6.25
51677,579885,23452,HEART MINI PORTRAIT FRAME,4,2011-11-30 17:37:00,1.95,15444.0,United Kingdom,7.80
51678,579885,23084,RABBIT NIGHT LIGHT,6,2011-11-30 17:37:00,2.08,15444.0,United Kingdom,12.48
51679,C579886,22594,CHRISTMAS GINGHAM TREE,-1,2011-11-30 17:39:00,0.85,15676.0,United Kingdom,-0.85


In [77]:
# количество заказов
len(df_2011_11)

8563

In [78]:
# количество клиентов
len(df_2011_11['CustomerID'].unique())

1375

In [79]:
# сумма дохода
df_2011_11.revenue.sum()

156472.6

### 2011-12

In [80]:
start_date = '2011-12-01 00:00:00'
df_2011_12_start_date = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = '2011-12-31 23:59:59'
df_2011_12_end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

In [81]:
df_2011_12 = df.loc[(df.InvoiceDate >= df_2011_12_start_date) & (df.InvoiceDate <= df_2011_12_end_date)]
df_2011_12

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,revenue
51681,C579890,84947,ANTIQUE SILVER TEA GLASS ENGRAVED,-1,2011-12-01 08:14:00,1.25,15197.0,United Kingdom,-1.25
51682,C579891,23485,BOTANICAL GARDENS WALL CLOCK,-1,2011-12-01 08:18:00,25.00,13644.0,United Kingdom,-25.00
51683,C579894,23568,EGG CUP HENRIETTA HEN CREAM,-1,2011-12-01 08:26:00,1.25,13098.0,United Kingdom,-1.25
51684,C579897,22470,HEART OF WICKER LARGE,-2,2011-12-01 08:29:00,2.95,17636.0,United Kingdom,-5.90
51685,579899,22623,BOX OF VINTAGE JIGSAW BLOCKS,3,2011-12-01 08:33:00,5.95,15687.0,United Kingdom,17.85
...,...,...,...,...,...,...,...,...,...
54209,581585,84832,ZINC WILLIE WINKIE CANDLE STICK,24,2011-12-09 12:31:00,0.85,15804.0,United Kingdom,20.40
54210,581585,23084,RABBIT NIGHT LIGHT,12,2011-12-09 12:31:00,2.08,15804.0,United Kingdom,24.96
54211,581585,84692,BOX OF 24 COCKTAIL PARASOLS,25,2011-12-09 12:31:00,0.42,15804.0,United Kingdom,10.50
54212,581587,22631,CIRCUS PARADE LUNCH BOX,12,2011-12-09 12:50:00,1.95,12680.0,France,23.40


In [82]:
# количество заказов
len(df_2011_12)

2533

In [83]:
# количество клиентов
len(df_2011_12['CustomerID'].unique())

493

In [84]:
# сумма дохода
df_2011_12.revenue.sum()

48960.950000000004

## Выводы и рекомендации

- Предполагаемая цель бизнеса: расширение географии продаж и увеличение дохода в странах кроме UK

- Возможная боль бизнеса: есть возможность масштабироваться и увеличивать количество заказов из разных стран, но непонятно в развитие в каких странах нужно вкладываться в первую очередь. Есть много стран, из которых поступают заказы, но основной доход приносят заказы из UK.

- Заказчиком может быть директор по развитию, коммерческий директор, ген директор

- На дашборд вынес средний чек, количество заказов, суммарный доход в разрезе по странам. Исходя из этого можно предположить в каких странах имеет смысл в первую очередь вкладываться в продвижение и увеличение продаж. 
https://datastudio.google.com/reporting/90800ef9-0a40-4c58-b548-b006d6d5eafb

- Рекомендации. Провести более детальное изучение таких регионов продаж, как: Германия, Ирландия и Австралия. Это регионы, в которых у нас больше всего клиентов, заказов и самый высокий средний чек. Надо узнать какие выходят стоимость клиента и стоимость заказа, какие категории товаров больше всего заказывают и какая маржинальность у этих товаров. Исходя из этих критерией можно выбрать приоритетную страну для дальнейшего 

- Также на основе имеющихся данных можно провести RFM-анализ. С помощью этого мы разобьем нашу базу клиентов на сегменты. После этого можем выстраивать разные маркетинговые коммуникации для каждого сегмента. Заказчиком выступает отдел маркетинга. Можем предоставить им эти сегменты, но для выведения на дашборд эта информация не подходит 

- На дашборд также вынес показатель дохода в динамике по месяцам. Заказчиком может выступать топ-менеджемт. В том числе при необходимости можно вынести на дашборд динамику в разрезе по месяцам по количеству клиентов и количетву заказов